In [2]:
pip install pandas psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 8.4 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import psycopg2 as pg

engine = pg.connect("dbname='clever' user='clever' host='postgres_clever' port='5432' password='clever'")

In [ ]:
## Introduction

### The process I used for solving this data trial was running all the queries in pgAdmin before documenting the final queries here.
### This allowed me to visualize and validate data formats, categories inside columns and keys for joining tables.
### 

In [ ]:
## Google Maps Company Profiles Data

In [8]:
company_profiles = pd.read_sql(
    """
    with sourcing as (
    	select
    		google_id
    		, name as company_name
    		, site as company_website
    		, type
    		, subtypes
    		, category
    		, phone
    		, full_address
    		, borough
    		, street
    		, city
    		, postal_code
    		, state
    		, country
    		, country_code
    		, latitude::float as latitude
    		, longitude::float as longitude
    		, time_zone
    		, plus_code -- google maps address for people that don't have one
    		, area_service
    		, rating::float as rating
    		, reviews_link
    		, reviews_tags
    		, photo
    		, photos_count -- review the necessity of this info
    		, street_view
    		, located_in
    		, regexp_replace(regexp_replace(working_hours, '\\\\', '', 'g'), '&&', '"', 'g') as working_hours
    		, popular_times -- mostly null values
    		, case
    			when business_status = 'CLOSED_PERMANENTLY'
    				then 'Closed Permanently'
    			when business_status = 'CLOSED_TEMPORARILY'
    				then 'Closed Temporarily'
    			when business_status = 'OPERATIONAL'
    				then 'Operational'
    			else null
    		end as business_status
    		, regexp_replace(regexp_replace(about, '\\\\', '', 'g'), '&&', '"', 'g') as about
    		, range -- mostly null
    		-- , posts -- mostly null
    		, logo
    		, description -- mostly null
    		, typical_time_spent -- mostly_null
    		, verified::boolean as verified
    		, owner_id::varchar as owner_id
    		, owner_title
    		, owner_link
    		, reservation_links
    		, booking_appointment_link
    		, menu_link
    		, order_links
    		, location_link
    		, place_id
    		, cid::varchar as cid
    		, reviews_id::varchar as reviews_id
    		, located_google_id -- mostly null
    		, reviews -- mostly null
    	from public.company_profiles_escaped
    )
    , extracting_json as ( 
    	select
            google_id
    		, company_name
    		, company_website
    		, type
    		, subtypes
    		, category
    		, phone
    		, full_address
    		, borough
    		, street
    		, city
    		, postal_code
    		, state
    		, country
    		, country_code
    		, latitude
    		, longitude
    		, time_zone
    		, plus_code
    		, area_service
    		, rating
    		, reviews_link
    		, reviews_tags
    		, photo
    		, photos_count
    		, street_view
    		, located_in
    		/* Working Hours */
    		, coalesce(regexp_replace(cast(working_hours::json->'Sunday' as varchar), '"', '', 'g'), 'Not informed') as sunday_working_hours
    		, coalesce(regexp_replace(cast(working_hours::json->'Monday' as varchar), '"', '', 'g'), 'Not informed') as monday_working_hours
            , coalesce(regexp_replace(cast(working_hours::json->'Tuesday' as varchar), '"', '', 'g'), 'Not informed') as tuesday_working_hours
            , coalesce(regexp_replace(cast(working_hours::json->'Wednesday' as varchar), '"', '', 'g'), 'Not informed') as wednesday_working_hours
            , coalesce(regexp_replace(cast(working_hours::json->'Thursday' as varchar), '"', '', 'g'), 'Not informed') as thursday_working_hours
            , coalesce(regexp_replace(cast(working_hours::json->'Friday' as varchar), '"', '', 'g'), 'Not informed') as friday_working_hours
            , coalesce(regexp_replace(cast(working_hours::json->'Saturday' as varchar), '"', '', 'g'), 'Not informed') as saturday_working_hours
    		, popular_times
    		, business_status
    		/* Sefice Options */
    		, coalesce(about::json->'Service options'->>'Curbside pickup', 'false')::boolean as curbside_pickup
            , coalesce(about::json->'Service options'->>'Delivery', 'false')::boolean as delivery
            , coalesce(about::json->'Service options'->>'Dine-in', 'false')::boolean as dine_in
            , coalesce(about::json->'Service options'->>'In-store pickup', 'false')::boolean as in_store_pickup
            , coalesce(about::json->'Service options'->>'In-store shopping', 'false')::boolean as in_store_shopping
    		, coalesce(about::json->'Service options'->>'Language assistance', 'No') AS language_assistance
            , coalesce(about::json->'Service options'->>'No-contact delivery', 'false')::boolean as no_contact_delivery
            , coalesce(about::json->'Service options'->>'Online appointments', 'false')::boolean as online_appointments
            , coalesce(about::json->'Service options'->>'Online classes', 'false')::boolean as online_classes
            , coalesce(about::json->'Service options'->>'Online estimates', 'false')::boolean as online_estimates
            , coalesce(about::json->'Service options'->>'Onsite services', 'false')::boolean as onsite_services
            , coalesce(about::json->'Service options'->>'Same-day delivery', 'false')::boolean as same_day_delivery
            , coalesce(about::json->'Service options'->>'Takeout', 'false')::boolean as takeout
    		/* Accessibility */
            , coalesce(about::json->'Accessibility'->>'Wheelchair accessible elevator', 'false')::boolean as wheelchair_accessible_elevator
            , coalesce(about::json->'Accessibility'->>'Wheelchair accessible entrance', 'false')::boolean as wheelchair_accessible_entrance
            , coalesce(about::json->'Accessibility'->>'Wheelchair accessible parking lot', 'false')::boolean as wheelchair_accessible_parking_lot
            , coalesce(about::json->'Accessibility'->>'Wheelchair accessible restroom', 'false')::boolean as wheelchair_accessible_restroom
            , coalesce(about::json->'Accessibility'->>'Wheelchair accessible seating', 'false')::boolean as wheelchair_accessible_seating
            /* From the business */
    		, coalesce(about::json->'From the business'->>'Identifies as Asian-owned', 'false')::boolean as identifies_as_asian_owned
            , coalesce(about::json->'From the business'->>'Identifies as Black-owned', 'false')::boolean as identifies_as_black_owned
            , coalesce(about::json->'From the business'->>'Identifies as Indigenous-owned', 'false')::boolean as identifies_as_indigenous_owned
            , coalesce(about::json->'From the business'->>'Identifies as LGBTQ+ owned', 'false')::boolean as identifies_as_lgbtq_owned
            , coalesce(about::json->'From the business'->>'Identifies as Latino-owned', 'false')::boolean as identifies_as_latino_owned
            , coalesce(about::json->'From the business'->>'Identifies as disabled-owned', 'false')::boolean as identifies_as_disabled_owned
            , coalesce(about::json->'From the business'->>'Identifies as veteran-owned', 'false')::boolean as identifies_as_veteran_owned
            , coalesce(about::json->'From the business'->>'Identifies as women-owned', 'false')::boolean as identifies_as_women_owned
            , coalesce(about::json->'From the business'->>'Small business', 'false')::boolean as small_business
            /* Crowd */
    		, coalesce(about::json->'Crowd'->>'Groups', 'false')::boolean as groups
            , coalesce(about::json->'Crowd'->>'LGBTQ+ friendly', 'false')::boolean as lgbtq_friendly
            , coalesce(about::json->'Crowd'->>'Tourists', 'false')::boolean as tourists
            , coalesce(about::json->'Crowd'->>'Transgender safespace', 'false')::boolean as transgender_safespace
            /* Meeting Planning */
    		, coalesce(about::json->'Planning'->>'Accepts reservations', 'false')::boolean as accepts_reservations
            , coalesce(about::json->'Planning'->>'Appointment required', 'false')::boolean as appointment_required
            , coalesce(about::json->'Planning'->>'Appointments recommended', 'false')::boolean as appointments_recommended
            , coalesce(about::json->'Planning'->>'Quick visit', 'false')::boolean as quick_visit
            /* Payment Methods */
    		, coalesce(about::json->'Payments'->>'Checks', 'false')::boolean as payments_checks
            , coalesce(about::json->'Payments'->>'Credit cards', 'false')::boolean as payments_credit_cards
            , coalesce(about::json->'Payments'->>'Debit cards', 'false')::boolean as payments_debit_cards
            , coalesce(about::json->'Payments'->>'NFC mobile payments', 'false')::boolean as payments_nfc_mobile
            , coalesce(about::json->'Payments'->>'SNAP/EBT', 'false')::boolean as payments_snap_ebt
    		, logo
    		, description -- mostly null
    		, typical_time_spent -- mostly_null
    		, verified
    		, owner_id
    		, owner_title
    		, owner_link
    		, reservation_links
    		, booking_appointment_link
    		, menu_link
    		, order_links
    		, location_link
    		, place_id
    		, cid
    	from sourcing
    )
    select *
    from extracting_json
    """
    , con=engine)
print(company_profiles)

/tmp/ipykernel_3098/1842622984.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  company_profiles = pd.read_sql(


                                  google_id  \
0      0x111f53bf849f031:0xb43eff4f62923c5d   
1      0x14e179155727173b:0x2c342e5f13a9b2d   
2     0x14e4b9f6c46524a9:0x202749ab2be6bfe3   
3       0x1803389f251373:0x26e9e6cc7312627c   
4      0x18751cf7cb7e9a5:0xad85fc0dfe9ff490   
...                                     ...   
5860  0x88d9b526dc1dde15:0xf1a594977f12ec6c   
5861   0xcce2349dd8997e9:0xd68bae2d806e12fb   
5862  0x864e9965507a75ff:0xf3b4cbcb39e97acf   
5863   0xcd01879723234e5:0x3ea7f9f4c8a4adb7   
5864   0xdd29bbfc28930ed:0x2f32238a396b70bf   

                                           company_name  \
0                                   Nicolas Boucher P.A   
1                                       New Hope Realty   
2                            Harvey Mclean & Associates   
3                                    Yanet Mino Realtor   
4            Lunabase Travelstays & Property Management   
...                                                 ...   
5860                  

In [12]:
company_profiles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5865 entries, 0 to 5864
Data columns (total 90 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   google_id                          5865 non-null   object 
 1   company_name                       5865 non-null   object 
 2   company_website                    4499 non-null   object 
 3   type                               5858 non-null   object 
 4   subtypes                           5858 non-null   object 
 5   category                           5858 non-null   object 
 6   phone                              5622 non-null   object 
 7   full_address                       5865 non-null   object 
 8   borough                            3823 non-null   object 
 9   street                             5845 non-null   object 
 10  city                               5865 non-null   object 
 11  postal_code                        5864 non-null   objec

0                           Nicolas Boucher P.A
1                               New Hope Realty
2                    Harvey Mclean & Associates
3                            Yanet Mino Realtor
4    Lunabase Travelstays & Property Management
5                     Nathalie Alfonso, Realtor
6                             Patricia Robinson
7            Jose Luis Guilarte - Miami Realtor
8                     Hector Rios, PA | Realtor
9                    Trace Real Estate Services
Name: company_name, dtype: object

In [13]:
### Testing if the transformations worked properly, especially the json extract ones.
company_profiles[['company_name', 'monday_working_hours', 'wheelchair_accessible_entrance']].head(10)

,company_name,monday_working_hours,wheelchair_accessible_entrance
0,Nicolas Boucher P.A,9AM-8PM,True
1,New Hope Realty,9AM-5PM,False
2,Harvey Mclean & Associates,Not informed,False
3,Yanet Mino Realtor,Open 24 hours,True
4,Lunabase Travelstays & Property Management,Open 24 hours,True
5,"Nathalie Alfonso, Realtor",Open 24 hours,True
6,Patricia Robinson,9AM-9PM,True
7,Jose Luis Guilarte - Miami Realtor,Open 24 hours,True
8,"Hector Rios, PA | Realtor",9AM-9PM,True
9,Trace Real Estate Services,Not informed,True


In [ ]:
## FMCSA Data

### The first step was to explore the FMCSA data. By doing this, I found that fmcsa_companies,
### fmcsa_company_snapshot and fmcsa_safer_data have the same grain, which is the company (usdot_num) itself.
### The three datasets have similar information, ones having the same columns and others having more columns than others.
### This was enought to realise they can be joined in only one column to make it easier to dispose the information to the client.
### By doing the query below, it's possible to understand if the keys (later PK and FKs) join correctly and also show which usdot_nums are missing in the safer_data dataset

In [5]:
fmcsa_data = pd.read_sql(
    """
    select
    	fmcsa_companies.usdot_num
    	, fmcsa_company_snapshot.usdot_num
    	, fmcsa_safer_data.usdot_num
    from public.fmcsa_companies
    left join public.fmcsa_company_snapshot
    	on fmcsa_companies.usdot_num = fmcsa_company_snapshot.usdot_num
    left join public.fmcsa_safer_data
    	on fmcsa_companies.usdot_num = fmcsa_safer_data.usdot_num
    where fmcsa_companies.usdot_num is null
    	or fmcsa_company_snapshot.usdot_num is null
    	or fmcsa_safer_data.usdot_num is null
    """
, con=engine)
print(fmcsa_data)

  usdot_num usdot_num usdot_num
0   3035106   3035106      None
1   3535426   3535426      None
2   3148696   3148696      None
3   3661270   3661270      None
4   3166409   3166409      None
5   3722904   3722904      None


/tmp/ipykernel_3098/2846959786.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  fmcsa_data = pd.read_sql(


In [ ]:
### With this I could write the staging layers to later join them in a fact_fmcsa_companies. I decided to consider it a fact table
### because it has quantitative information about the companies and, by definition, those are facts.

In [6]:
fact_fmcsa_companies = pd.read_sql(
    """
    with companies as (
    	select
    		usdot_num
    		, user_created
    		, date_created
    		, user_updated
    		, date_updated
    		, company_name
    		, company_url
    		, city
    		, state
    		, location
    		, company_type
    		, total_complaints_2021
    		, total_complaints_2022
    		, total_complaints_2023
    	from public.fmcsa_companies
    )
    , company_snapshot as (
    	select
    		usdot_num
    		, fmsca_ai_profile as fmcsa_ai_profile
    		, user_created
    		, date_created
    		, user_updated
    		, date_updated
    		, company_name
    		, mc_num
    		, registered_address
    		, case
    			when mailing_address = 'Same as above'
    				then registered_address
    			else mailing_address
    		end as mailing_address
    		, phone_number
    		, fax_number
    		, safety_review_date
    		, num_of_trucks as number_of_trucks
    		, num_of_tractors as number_of_tractors
    		, num_of_trailers as number_of_trailers
    		, hhg_authorization
    		, total_complaints_2021
    		, total_complaints_2022
    		, total_complaints_2023
    	from public.fmcsa_company_snapshot
    )
    , safer_data as (
    	select
    	    usdot_num
    		, fmcsa_link
    		, user_created
    		, date_created
    		, user_updated
    		, date_updated
    		, entity_type
    		, operating_status
    		, oos_date
    		, legal_name
    		, dba_name
    		, physical_address
    		, phone
    		, mailing_address
    		-- , state_carrier_id_number -- null
    		, mc_num
    		, duns_number
    		, power_units
    		, coalesce(drivers, 0) as total_drivers
    		, mcs_150_form_date -- form
    		, operation_classification
    		, carrier_type
    	    , case
    	        when cargo_types like '%General Freight%'
    	            then 'True'
    	        else 'False'
    	    end as general_freight_transport
    	    , case
    	        when cargo_types like '%Household Goods%'
    	            then 'True'
    	        else 'False'
    	    end as household_goods_transport
    	    , case
    	        when cargo_types like '%Furniture%'
    	            then 'True'
    	        else 'False'
    	    end as furniture_transport
    		, case
    	        when cargo_types like '%FINE ART%'
    	            then 'True'
    	        else 'False'
    	    end as fine_art_transport
    		, us_vehicle_inspections
    	    , us_driver_inspections
    	    , us_hazmat_inspections
    	    , us_iep_inspections
    	    , us_vehicle_out_of_service
    	    , us_driver_out_of_service
    	    , us_hazmat_out_of_service
    	    , us_iep_out_of_service
    	    , us_vehicle_out_of_service_pct
    	    , us_driver_out_of_service_pct
    	    , us_hazmat_out_of_service_pct
    	    , us_iep_out_of_service_pct
    	    , us_vehicle_natl_avg_oos_pct
    	    , us_driver_natl_avg_oos_pct
    	    , us_hazmat_natl_avg_oos_pct
    	    , us_iep_natl_avg_oos_pct
    	    , us_crashes_fatal
    	    , us_crashes_injury
    	    , us_crashes_tow
    	    , us_crashes_total
    	    , canadian_vehicle_inspections
    	    , canadian_driver_inspections
    	    , canadian_vehicle_out_of_service
    	    , canadian_driver_out_of_service
    	    , canadian_vehicle_out_of_service_pct
    	    , canadian_driver_out_of_service_pct
    	    , carrier_safety_rating_rating_date
    	    , carrier_safety_rating_review_date
    	    , carrier_safety_rating_rating
    	    , carrier_safety_rating_type
    	    , mileage -- cant coalesce 0 here 
    	    , mileage_year
    	from public.fmcsa_safer_data
    )
    , joining as (
    	select
    		companies.usdot_num
    		, companies.company_name
    		, safer_data.dba_name
    		
    		, companies.city
    		, companies.state
    		, companies.location
    		, companies.company_type
    		, companies.total_complaints_2021
    		, companies.total_complaints_2022
    		, companies.total_complaints_2023
    
    		, company_snapshot.mc_num
    		, coalesce(company_snapshot.registered_address, safer_data.physical_address) as company_address
    		, coalesce(company_snapshot.mailing_address, safer_data.mailing_address) as mailing_address
    		, company_snapshot.phone_number
    		, company_snapshot.fax_number
    		
    		, company_snapshot.number_of_trucks
    		, company_snapshot.number_of_tractors
    		, company_snapshot.number_of_trailers
    		, safer_data.power_units
    		, safer_data.total_drivers
    		-- , safer_data.duns_number
    		
    		, safer_data.mcs_150_form_date -- form
    		, safer_data.operation_classification
    		, safer_data.entity_type
    		, safer_data.carrier_type -- explain it in the report
    		/* trasnport type */
    		, coalesce(company_snapshot.hhg_authorization, false) as hhg_authorization
    		, safer_data.general_freight_transport::boolean
    		, safer_data.household_goods_transport::boolean
    		, safer_data.furniture_transport::boolean
    		, safer_data.fine_art_transport::boolean
    		/* vehicle and driver information */
    		, safer_data.us_vehicle_inspections
    	    , safer_data.us_driver_inspections
    	    , safer_data.us_hazmat_inspections
    	    , safer_data.us_iep_inspections
    	    , safer_data.us_vehicle_out_of_service
    	    , safer_data.us_driver_out_of_service
    	    , safer_data.us_hazmat_out_of_service
    	    , safer_data.us_iep_out_of_service
    	    , safer_data.us_vehicle_out_of_service_pct
    	    , safer_data.us_driver_out_of_service_pct
    	    , safer_data.us_hazmat_out_of_service_pct
    	    , safer_data.us_iep_out_of_service_pct
    	    , safer_data.us_vehicle_natl_avg_oos_pct
    	    , safer_data.us_driver_natl_avg_oos_pct
    	    , safer_data.us_hazmat_natl_avg_oos_pct
    	    , us_iep_natl_avg_oos_pct
    	    , safer_data.us_crashes_fatal
    	    , safer_data.us_crashes_injury
    	    , safer_data.us_crashes_tow
    	    , safer_data.us_crashes_total
    	    , safer_data.canadian_vehicle_inspections
    	    , safer_data.canadian_driver_inspections
    	    , safer_data.canadian_vehicle_out_of_service
    	    , safer_data.canadian_driver_out_of_service
    	    , safer_data.canadian_vehicle_out_of_service_pct
    	    , safer_data.canadian_driver_out_of_service_pct
    	    , safer_data.carrier_safety_rating_rating_date
    	    , safer_data.carrier_safety_rating_review_date
    	    , safer_data.carrier_safety_rating_rating
    	    , safer_data.carrier_safety_rating_type
    	    , safer_data.mileage -- cant coalesce 0 here 
    	    , safer_data.mileage_year
    
    		/* urls */
    		, companies.company_url
    		, company_snapshot.fmcsa_ai_profile
    		, safer_data.fmcsa_link
    		/* metadata */
    		, companies.user_created
    		, companies.date_created
    		, companies.user_updated
    		, companies.date_updated
    	from companies
    left join company_snapshot
    	on companies.usdot_num = company_snapshot.usdot_num
    left join safer_data
    	on companies.usdot_num = safer_data.usdot_num
    )
    select *
    from joining
    """
    , con=engine)
print(fact_fmcsa_companies)

     usdot_num                       company_name         dba_name    city  \
0      1032910                    FANTASTIC MOVES  FANTASTIC MOVES  DALLAS   
1      1151019               TOP 10 VAN LINES INC             None   MIAMI   
2      1170441      GENESIS MOVING & STORAGE,INC.             None   MIAMI   
3      1174864                      ALEXIM MOVING             None   MIAMI   
4      1182931           E J R TRANSPORTATION INC             None   MIAMI   
..         ...                                ...              ...     ...   
395    3535426      CARRERA'S TRUCKS AND DELIVERY             None   MIAMI   
396    3148696       OCONNOR FREIGHT COMPANY CORP             None   MIAMI   
397    3661270         A. BRADLEY ENTERPRISES LLC             None  DALLAS   
398    3166409  MOTOR CARRIER OF HOUSE HOLD GOODS             None  DALLAS   
399    3722904                    CCBS MOTORS LLC             None   MIAMI   

    state    location  company_type  total_complaints_2021  \
0

/tmp/ipykernel_3098/674014953.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  fact_fmcsa_companies = pd.read_sql(


In [11]:
### After that, I built the fact_fmcsa_complaints. I decided not to join this table on fact_fmcsa_companies because they have different granularities
### By having two tables, we maintain data consistency and flexibility (normalized).
### Exploring the data first:

In [13]:
complaints_data = pd.read_sql(
    """
    select
    	complaint_category
    	, count(*) as counting
    from public.fmcsa_complaints
    group by complaint_category
    order by counting desc
    """
    , con=engine)
print(complaints_data)

                                  complaint_category  counting
0                            Estimates/Final Charges        63
1   Consumer Complaint(Deceptive Business Practices)        62
2                                    Loss and Damage        59
3                                 Shipment Documents        58
4                                Pickup and Delivery        55
5                                   Claim Settlement        54
6                                           Weighing        47
7                                            Hostage        40
8                                Operating Authority        18
9                                          Insurance         9
10                       Other Commercial Complaints         6
11               Owner Operator - Leasing Violations         2
12                           Whistleblower Complaint         1


/tmp/ipykernel_20839/1766831061.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  complaints_data = pd.read_sql(


In [14]:
complaints_year = pd.read_sql(
    """
    select
    	complaint_year
    	, count(*) as counting
    from public.fmcsa_complaints
    group by complaint_year
    order by counting desc
    """
    , con=engine)
print(complaints_year)

   complaint_year  counting
0            2022       188
1            2021       167
2            2023       119


/tmp/ipykernel_20839/1785847772.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  complaints_year = pd.read_sql(


In [15]:
### Joining fmcsa_companies on fmcsa_complaint so we can visualize the companies' name and filter it by city/state:

In [17]:
fact_fmcsa_complaints = pd.read_sql(
    """
    select
    	complaints.usdot_num
    	, complaints.id as complaint_id
    	, companies.company_name
    	, companies.city
    	, companies.state
    	, companies.location
    	, complaints.complaint_category
    	, complaints.complaint_year
    	, complaints.complaint_count
    	/* metadata */
    	, complaints.user_created
    	, complaints.date_created
    	, complaints.user_updated --null
    	, complaints.date_updated --null
    from public.fmcsa_complaints as complaints
    inner join public.fmcsa_companies as companies
    	on complaints.usdot_num = companies.usdot_num
    """
    , con=engine)
fact_fmcsa_complaints.head(10)

/tmp/ipykernel_20839/18103829.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  fact_fmcsa_complaints = pd.read_sql(


,usdot_num,complaint_id,company_name,city,state,location,complaint_category,complaint_year,complaint_count,user_created,date_created,user_updated,date_updated
0,3500694,003e5853-e612-4661-b889-1113a62212ba,ALPHA RELOCATIONS LLC,DALLAS,TX,"DALLAS, TX",Consumer Complaint(Deceptive Business Practices),2021,1,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T17:59:52.992Z,None,None
1,3179208,0164fac5-9ed5-49e2-a60b-874a3a4e5760,ALL MY SONS OF DALLAS SOUTH LLC,DALLAS,TX,"DALLAS, TX",Estimates/Final Charges,2021,1,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T17:59:52.929Z,None,None
2,2543958,018d14dc-a7d3-438d-8162-0acdfc926806,JACOB AND SONS MOVING & STORAGE,MIAMI,FL,"MIAMI, FL",Consumer Complaint(Deceptive Business Practices),2022,7,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T18:00:14.278Z,None,None
3,2955542,01bf5740-de67-4a41-a971-9266923a11a1,OUT STATE MOVERS,DALLAS,TX,"DALLAS, TX",Pickup and Delivery,2022,2,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T17:59:53.099Z,None,None
4,3307068,027b31c6-6493-47f2-82ca-92e223a7558b,US STANDARD MOVING & STORAGE CORP,MIAMI,FL,"MIAMI, FL",Weighing,2022,2,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T18:00:14.690Z,None,None
5,3307068,027fec86-cece-4c3c-99b4-d46c941dd16a,US STANDARD MOVING & STORAGE CORP,MIAMI,FL,"MIAMI, FL",Shipment Documents,2022,2,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T18:00:14.684Z,None,None
6,2372043,029822b2-9f8b-4e76-9338-a7c5ad5cbfb8,COACH USA MOVING AND STORAGE,MIAMI,FL,"MIAMI, FL",Operating Authority,2021,1,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T18:00:14.090Z,None,None
7,1664920,03425935-4588-4d81-a9a6-a96975d02b52,IDEAL MOVING AND STORAGE INC,DALLAS,TX,"DALLAS, TX",Loss and Damage,2023,1,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T17:59:53.043Z,None,None
8,2955542,039aaed0-ad10-4f83-ad07-065f4bc955ef,OUT STATE MOVERS,DALLAS,TX,"DALLAS, TX",Estimates/Final Charges,2022,2,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T17:59:53.090Z,None,None
9,3165347,04c7f987-357d-48c5-a06d-f9879736be07,SHYFT MOVING,MIAMI,FL,"MIAMI, FL",Consumer Complaint(Deceptive Business Practices),2023,1,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T18:00:14.509Z,None,None


In [ ]:
 = pd.read_sql(
    """
    """
    , con=engine)
print()

In [ ]:
 = pd.read_sql(
    """
    """
    , con=engine)
print()